<a href="https://colab.research.google.com/github/iued-uni-heidelberg/corpustools/blob/main/S101lemHYv202511RB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rule-based lemmatization for Armenian

## Stage 1: Downloading example file
Example: Universal declaration of human rights

If downloading from the UN website, the link aboout pdf2txt conversion is https://chatgpt.com/share/68bbeed8-0d08-800e-8418-b0816c6393d4



In [ ]:
%%bash
wget https://heibox.uni-heidelberg.de/f/7a091a3c0372428a9aa2/?dl=1
mv index.html?dl=1 udhr-all-languages.zip
unzip udhr-all-languages.zip

In [17]:
%%bash

# Downloading UDHR
# !wget https://unicode.org/udhr/assemblies/udhr_txt.zip
# Alternatively, downloading from the UN website, converting pdf to txt
# the link aboout pdf2txt conversion: https://chatgpt.com/share/68bbeed8-0d08-800e-8418-b0816c6393d4
# !rm --recursive udhr-all-languages

# delete lines which are not translations in some files (hy)
# delete between lines $a and $b inclusive
a=9
b=21
awk -v m=$a -v n=$b 'm <= NR && NR <= n {next} {print}' < /content/udhr-all-languages/udhr_hye.txt >/content/udhr-all-languages/udhr_hye_v02.txt

a=1
b=6
awk -v m=$a -v n=$b 'm <= NR && NR <= n {next} {print}' < /content/udhr-all-languages/udhr_hye_v02.txt >/content/udhr-all-languages/udhr_hye_v03.txt



# put paragraph tags
# awk '{print "<p>\n"$0 ; print "</p>"}' udhr/udhr_hye2.txt >udhr/udhr_hye_v03.txt


cp /content/udhr-all-languages/udhr_hye_v03.txt /content/udhr_hye_v03.txt
head --lines=10 /content/udhr_hye_v03.txt


ՄԱՐԴՈՒ ԻՐԱՎՈՒՆՔՆԵՐԻ ՀԱՄԸՆԴՀԱՆՈՒՐ ՀՌՉԱԿԱԳԻՐ
    ՄԻԱՎՈՐՎԱԾ ԱԶԳԵՐԻ ԿԱԶՄԱԿԵՐՊՈՒԹՅՈՒՆ
    ՆԵՐԱԾԱԿԱՆ
    Քանզի մարդկային ընտանիքի բոլոր անդամներին ներհատուկ արժանապատվությունը և հավասար ու անօտարելի իրավունքները աշխարհի ազատության, արդարության ու խաղաղության հիմքն են․
    Քանզի մարդու իրավունքների նկատմամբ քամահրանքն ու արհամարհանքը հանգեցրել են մարդկության խիղճը խռոված բարբարոսական գործողությունների, և քանի որ այնպիսի աշխարհի ստեղծումը, ուր մարդիկ կվայելեն խոսքի ու համոզմունքների ազատություն և զերծ կլինեն վախից ու կարիքից հռչակվել է որպես մարդկանց բարձրագույն ձգտում․
    Քանզի անհրաժեշտ է, որպեսզի մարդը, որպես մի վերջին միջոցի, չդիմի ապստամբության ընդդեմ բռնության ու ճնշման, օրենքի իշխանությամբ պաշտպանել մարդու իրավունքները․
    Քանզի անհրաժեշտ է նպաստել ազգերի միջև բարեկամական հարաբերությունների զարգացմանը․
    Քանզի Միավորված ազգերի ժողովուրդները կանոնադրության մեջ վերահավաստել են իրենց հավատը մարդու հիմնական իրավունքների, անձի արժանապատվության ու արժեքի, տղամարդու ու կնոջ հավասար իրավուն

## Installing packages

- gensim word2vec for similarity-based disambiguation
- Eastern Armenian Analyser from https://github.com/timarkh/uniparser-grammar-eastern-armenian , which outputs translations into English


Note, please restart the session if requested (gensim needs another version of the default libraries).



In [2]:
# installing Armenian morphological analyser
!git clone https://github.com/timarkh/uniparser-grammar-eastern-armenian
# Python classes
!pip3 install uniparser-eastern-armenian
# disambiguation
!sudo apt-get install cg3

!pip install gensim


Cloning into 'uniparser-grammar-eastern-armenian'...
remote: Enumerating objects: 181, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 181 (delta 12), reused 40 (delta 12), pack-reused 141 (from 1)
Receiving objects: 100% (181/181), 52.66 MiB | 10.30 MiB/s, done.
Resolving deltas: 100% (78/78), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 3.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcg3-1
The following NEW packages will be installed:
  cg3 libcg3-1
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 423 kB of archives.
After this operation, 1,278 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcg3-1 amd64 1.3.2-1build2 

In [1]:
# importing python libraries
import os, re, sys

from uniparser_eastern_armenian import EasternArmenianAnalyzer
a = EasternArmenianAnalyzer()
analyses = a.analyze_words('Ձևաբանություն')
for ana in analyses:
    print(ana.wf, ana.lemma, ana.gramm, ana.gloss, ana.stem, ana.subwords, ana.wfGlossed, ana.otherData)

Ձևաբանություն ձեւաբանություն N,inanim,sg,nom,nonposs morphology ձևաբանություն. [] ձևաբանություն [('trans_en', 'morphology')]


## Optional part: trying different output format with disambiguation, etc.

This can be skipped. You can go directly to the "File annotation" section

In [ ]:
# trying out:
# nonexisting word
analyses2 = a.analyze_words('Ձևաբայու')
for ana2 in analyses2:
    if ana2.lemma:
      print(ana2.wf, ana2.lemma, ana2.gramm, ana2.gloss, ana2.stem, ana2.subwords, ana2.wfGlossed, ana2.otherData)
    else:
      print(ana2.wf, ana2.wf, "N", "x", ana2.stem, ana2.subwords, ana2.wfGlossed, ana2.otherData)

In [ ]:
analyses = a.analyze_words([['և'], ['Ես', 'սիրում', 'եմ', 'քեզ', ':']],
                           format='xml')
for ana in analyses:
    print(str(ana))

In [ ]:
analyses = a.analyze_words(['Ձևաբանություն', [['և'], ['Ես', 'սիրում', 'եմ', 'քեզ', ':']]],
                           format='json')
for ana in analyses:
    print(str(ana))

In [ ]:
# analysis with disambiguation
analyses = a.analyze_words(['Ես', 'սիրում', 'եմ', 'քեզ'], disambiguate=True)
for ana in analyses:
    if len(ana) > 1: tab = "  "
    else: tab = ""
    for wfo in ana:
        print(tab, wfo.wf, wfo.lemma, wfo.gramm, wfo.gloss)

Str = "Սառը, վճիտ ապրիլյան օր էր, ու ժամացույցը խփում էր տասներեքը։ Չար քամուց թաքնվելու համար կզակը սեղմելով կրծքին՝ Ուինսթոն Սմիթն արագ ներս խցկվեց «Հաղթանակ» բնակելի տան ապակե շքադռնից՝ իր ետևից ներս թողնելով հատիկավոր փոշու մի ամբողջ փոթորիկ։"

StrDe = ' „Es war ein kalter, trostloser Apriltag, und die Uhr schlug dreizehn. Das Kinn an die Brust gedrückt, um sich vor dem bitteren Wind zu schützen, eilte Winston Smith durch die gläserne Veranda des Wohnhauses Victory und hinterließ einen körnigen Sturm Staub." '

StrEn = ' "It was a cold, dreary April day, and the clock struck thirteen. Tucking his chin to his chest to shield himself from the bitter wind, Winston Smith hurried through the glass porch of the Victory apartment building, leaving behind him a storm of granular dust." '

In [5]:
Str = "Սառը, վճիտ. ապրիլյան օր էր, ու ժամացույցը խփում էր տասներեքը։ Չար քամուց թաքնվելու համար կզակը սեղմելով կրծքին՝ Ուինսթոն Սմիթն արագ ներս խցկվեց «Հաղթանակ» բնակելի տան ապակե շքադռնից՝ իր ետևից ներս թողնելով հատիկավոր փոշու մի ամբողջ փոթորիկ։"

In [6]:
Lst = re.split(r'([ ,\.:;\!\(\)\"\[\]՞՝«»\-\—՝։\։]+)', Str)
LstTok = []
for el in Lst:
    el = el.strip()
    if el != '': LstTok.append(el)


In [ ]:
print(LstTok)
# does disambiguation work? not yet...

In [ ]:
analyses = a.analyze_words(LstTok, disambiguate=True)
for ana in analyses:
    if len(ana) > 1: tab = "  "
    else: tab = ""
    for wfo in ana:
        print(tab, wfo.wf, wfo.lemma, wfo.gramm, wfo.gloss)

## File annotation

### defining functions to process the file

In [10]:
from gensim.models import Word2Vec # The word2vec model class
import gensim.downloader as api # Allows us to download some free training data

In [11]:
# Vahram's model for English
!wget https://heibox.uni-heidelberg.de/f/38de0eb8b2ec41d284d7/?dl=1
!mv index.html?dl=1 WIKI_EN.model


--2025-09-06 11:44:24--  https://heibox.uni-heidelberg.de/f/38de0eb8b2ec41d284d7/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/881c52e8-8f71-4c40-ad6d-9fa00342c7fb/WIKI_EN.model [following]
--2025-09-06 11:44:25--  https://heibox.uni-heidelberg.de/seafhttp/files/881c52e8-8f71-4c40-ad6d-9fa00342c7fb/WIKI_EN.model
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 90291014 (86M) [application/octet-stream]
Saving to: ‘index.html?dl=1’

index.html?dl=1     100%[===================>]  86.11M  13.1MB/s    in 8.0s    

2025-09-06 11:44:34 (10.7 MB/s) - ‘index.html?dl=1’ saved [90291014/90291014]



In [12]:
model_WIKI_EN = Word2Vec.load("/content/WIKI_EN.model")
word_vectors_WIKI_EN = model_WIKI_EN.wv

In [13]:
distance = word_vectors_WIKI_EN.similarity('obama', 'barak')
distance2 = word_vectors_WIKI_EN.similarity('obama', 'zone')
print('distance = %.4f' % distance)
print('distance2 = %.4f' % distance2)

distance = 0.7685
distance2 = 0.3015


In [14]:
def tokenizeHy(Str2tokenise, rePattern = r'([ ,\.:;\!\(\)\"\[\]՞՝«»\-\—՝։\։]+)'):
    LTokens = []
    Lst = re.split(rePattern, Str2tokenise)
    # LstTok = []
    for el in Lst:
        el = el.strip()
        if el != '': LTokens.append(el)
    return LTokens

In [15]:
# merging together different lines of code

def disambiguate(LTok2disambiguate, Window = 4, show_variants = False):
    SDisambig = ''

    analysesN = a.analyze_words(LTok2disambiguate, disambiguate=False)

    # preparing data structures
    LLContext = [] # empty list of contexts, indices are the same as with the Text list
    LLText = [] # test to disambiguate, ambiguous interpretations are double entries
    for ana in analysesN:
        # creating context window from glosses
        # if len(ana) > 1: tab = "~"
        # else: tab = "!"
        LwfoContext = []
        LwfoText = []
        for wfo in ana: # preserve lemmas / word forms which are not found in dictionary
            if wfo.gramm == '': wfo.gramm = 'N'
            if wfo.lemma == '': wfo.lemma = wfo.wf
            if wfo.gloss == '': wfo.gloss = '[unknown]'

            SWfo = f'{wfo.wf}\t{wfo.gramm}\t{wfo.lemma}\t{wfo.gloss}'
            # print(SWfo)
            # FNoD.write(SWfo + '\n')

            # find the first part of the gloss, which may be in the word vectors model
            REPart = re.match('([A-Za-z]+)', wfo.gloss)
            if REPart:
                SGlossMin = REPart.group(1)
                SGlossMin = SGlossMin.lower()
                # print(SGlossMin)
            else:
                SGlossMin = '[NONE]'
            LwfoContext.append(SGlossMin)
            LwfoText.append(SWfo)
        LLContext.append(LwfoContext)
        LLText.append(LwfoText)

    # for el in LLContext: print(el)
    # for el in LLText: print(el)

    # print(len(LLContext))
    # print(len(LLText))

    sys.stderr.write('.')


    for i in range(len(LLText)):
        if len(LLText[i]) > 1:
            # print(LLText[i])
            # print(LLContext[i])
            # collect context window +- 3 words
            iwStart = i-Window
            if iwStart <0: iwStart=0
            iwEnd = i+Window
            if iwEnd > len(LLText): iwEnd = len(LLText)
            # iwLen = iwEnd - iwStart
            winContext = LLContext[iwStart:iwEnd]
            # print(winContext)
            LScores = []
            LScCand = []

            for candidate in LLContext[i]:
                ScoreCand = 0
                for LCtx in winContext:
                    for Ctx in LCtx:
                        try: distance = word_vectors_WIKI_EN.similarity(candidate, Ctx)
                        except: distance = 0
                        ScoreCand += distance
                LScores.append((candidate,ScoreCand))
                LScCand.append(ScoreCand)
            LScores.sort(key=lambda a: a[1], reverse=True)
            # print(LScores)

            max_value = max(LScCand)
            #  Return the max value of the list
            max_index = LScCand.index(max_value)
            StoWrite = LLText[i][max_index] + '\n'
            SDisambig += StoWrite

            if show_variants == True:
                for el in LLText[i]:
                    StoWrite = '\t~\t' + el + '\n'
                    SDisambig += StoWrite
                for el in LScores:
                    StoWrite = '\t~sc:\t' + str(el) + '\n'
                    SDisambig += StoWrite
        else:
            StoWrite = LLText[i][0] + '\n'
            SDisambig += StoWrite

    return SDisambig

### give names for the input and output file

In [23]:

# FInText = open('/content/udhr/udhr_hye_v03.txt','r')
# FOutText = open('/content/udhrTT/udhr_hye_vert.txt','w')


FInText = open('udhr_hye_v03.txt','r')
FOutText = open('udhr_hye_v03.vert','w')

In [24]:
count = 0
for SLine in FInText:

    count += 1
    if count % 100 == 0: sys.stderr.write('\n')
    SLine = SLine.strip()
    LTok = tokenizeHy(SLine)
    # SDisambig = disambiguate(LTok, show_variants = True)
    SDisambig = disambiguate(LTok, show_variants = False)
    # FOutText.write('<p>\n')
    # FOutText.write('\n')
    FOutText.write(SDisambig)
    # FOutText.write('</p>\n')
    FOutText.write('\n')

FOutText.flush()

...................................................................................................
....................................................................................................
................

### optional: Creating a text file with lemmas

all word forms are converted into lemmas (might be used for training word2vec models)

In [26]:
# !awk -F '\t' '(NF==4){printf "%s ", $3}(NF!=4){printf "\n"}' < /content/udhrTT/udhr_hye_vert.txt >/content/udhrTT/udhr_hye_lem.txt
!awk -F '\t' '(NF==4){printf "%s ", $3}(NF!=4){printf "\n"}' < udhr_hye_v03.vert >udhr_hye_v03_lem.txt